In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(
    credential=DefaultAzureCredential(),
    path="config.json",
)

Found the config file in: config.json


In [34]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

model_name = "furlan-test-model"

model = Model(
    path="model.joblib",
    type=AssetTypes.CUSTOM_MODEL,
    name=model_name,
    description="furlan-test-model",
)

ml_client.models.create_or_update(model)

4


In [48]:
from azure.ai.ml.entities import Environment

env_name = "furlanp-test-environment"

env = Environment(
    name=env_name,
    conda_file="conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
)
ml_client.environments.create_or_update(env)

latest_env_version = max([
    int(e.version) for e in ml_client.environments.list(name=env_name)
])

print(latest_env_version)

10


by default azure takes
az login
az ml workspace update --name mlw-ki-ds-s --resource-group rg-ki-ds-s --image-build-compute FurlanP1

In [36]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

endpoint_name = "furlanp-test-endpoint"

endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="furlanp-test-endpoint",
    auth_mode="key",
)

endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

endpoint = ml_client.online_endpoints.get(name=endpoint_name)

print(
    f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)


Endpoint "furlanp-test-endpoint" with provisioning state "Succeeded" is retrieved


In [49]:
from azure.ai.ml.entities import ManagedOnlineDeployment
from azure.ai.ml.entities import CodeConfiguration

model_name = "furlan-test-model"
latest_model_version = max([int(m.version) for m in ml_client.models.list(name=model_name)])
model = ml_client.models.get(name=model_name, version=latest_model_version)

env_name = "furlanp-test-environment"
latest_env_version = max([int(e.version) for e in ml_client.environments.list(name=env_name)])
env = ml_client.environments.get(name=env_name, version=latest_env_version)

endpoint_name = "furlanp-test-endpoint"

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(".", "score.py"), 
    egress_public_network_access="disabled",
)

blue_deployment = ml_client.online_deployments.begin_create_or_update(blue_deployment).result()


Check: endpoint furlanp-test-endpoint exists
Uploading furlanp-azure-test (0.1 MBs): 100%|██████████| 97505/97505 [00:00<00:00, 201484.51it/s]




.....................................................................

In [55]:
endpoint_name = "furlanp-test-endpoint"
endpoint = ml_client.online_endpoints.get(name=endpoint_name)

print(endpoint.traffic, endpoint.scoring_uri)


ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name="blue",
    request_file="./sample_data.json",
)

logs = ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=endpoint_name, lines=50
)

print(logs)


{'blue': 0} https://furlanp-test-endpoint.westeurope.inference.ml.azure.com/score
Instance status:
SystemSetup: Succeeded
UserContainerImagePull: Succeeded
ModelDownload: Succeeded
UserContainerStart: Succeeded

Container events:
Kind: Pod, Name: Pulling, Type: Normal, Time: 2024-09-25T11:59:15.904736Z, Message: Start pulling container image
Kind: Pod, Name: Downloading, Type: Normal, Time: 2024-09-25T11:59:15.917266Z, Message: Start downloading models
Kind: Pod, Name: Pulled, Type: Normal, Time: 2024-09-25T11:59:37.495032Z, Message: Container image is pulled successfully
Kind: Pod, Name: Downloaded, Type: Normal, Time: 2024-09-25T11:59:37.495032Z, Message: Models are downloaded successfully
Kind: Pod, Name: Created, Type: Normal, Time: 2024-09-25T11:59:37.567938Z, Message: Created container inference-server
Kind: Pod, Name: Started, Type: Normal, Time: 2024-09-25T11:59:37.615229Z, Message: Started container inference-server
Kind: Pod, Name: ContainerReady, Type: Normal, Time: 2024-09-

In [74]:
res = ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name="blue",
    request_file="./sample_data.json",
)

In [75]:
import json

res = json.loads(res)

type(res)

list

In [76]:
import numpy as np
res = np.array(res)

print(res.shape)

(2, 15)


In [59]:
import requests

key = ml_client.online_endpoints.get_keys(endpoint_name).primary_key

res = requests.get(url=endpoint.scoring_uri, headers={"Authorization": f"Bearer {key}"})
print(res.content)



b'No valid deployments to route to. Please check that the endpoint has at least one deployment with positive weight values or use a deployment specific header to route. [https://docs.microsoft.com/en-us/azure/machine-learning/how-to-safely-rollout-managed-endpoints#deploy-a-new-model-but-send-it-no-traffic-yet]\nPlease check this guide to understand why this error code might have been returned \nhttps://docs.microsoft.com/en-us/azure/machine-learning/how-to-troubleshoot-online-endpoints#http-status-codes\n'
